# RQ2

To what extent can the choice of the vector DB diminish resource usage?

In [1]:
%load_ext autoreload
%autoreload 2

from results_processor import get_experiments_data, compute_total_energy_per_run, get_variation_runs_data, get_ci_deviation, compute_kruskal_wallis, compute_wilcoxon

### 1. Indexing 

In [2]:
indexing_experiments_data = get_experiments_data(
    "rq2_indexing",
    [
        "Total DRAM Power [W]",
        "IA Cores Power [W]",
    ],
)
indexing_runs_data = compute_total_energy_per_run(indexing_experiments_data)

In [3]:
index_db_variation_runs = get_variation_runs_data(
    indexing_runs_data,
    ["gte_base_cqadupstack_webmasters_milvus"],
    ["gte_base_cqadupstack_webmasters_qdrant"],
    ["gte_base_cqadupstack_webmasters_weaviate"],
    "database",
    ["milvus", "qdrant", "weaviate"],
)

In [4]:
index_db_variation_runs.groupby("database").aggregate(
    lambda x: str(round(x.mean(), 2)) + " ± " + str(get_ci_deviation(x))
)

,IA Cores Energy [J],DRAM Energy [J],duration [s]
database,,,
milvus,2625.1 ± 114.38,162.91 ± 7.08,132.04 ± 2.16
qdrant,3938.37 ± 75.24,288.41 ± 7.3,351.55 ± 18.21
weaviate,237.56 ± 5.34,18.67 ± 0.5,10.99 ± 0.19


In [5]:
compute_wilcoxon(index_db_variation_runs, "weaviate", "milvus", "IA Cores Energy [J]", "database")

WilcoxonResult(statistic=np.float64(0.0), pvalue=np.float64(0.0009765625))

In [6]:
compute_wilcoxon(index_db_variation_runs, "milvus", "qdrant", "IA Cores Energy [J]", "database")

WilcoxonResult(statistic=np.float64(0.0), pvalue=np.float64(0.0009765625))

In [7]:
compute_wilcoxon(index_db_variation_runs, "weaviate", "milvus", "duration [s]", "database")

WilcoxonResult(statistic=np.float64(0.0), pvalue=np.float64(0.0009765625))

In [8]:
compute_wilcoxon(index_db_variation_runs, "milvus", "qdrant", "duration [s]", "database")

WilcoxonResult(statistic=np.float64(0.0), pvalue=np.float64(0.0009765625))

### 2. Querying 

In [9]:
querying_experiments_data = get_experiments_data(
    "rq2_querying",
    [
        "Total DRAM Power [W]",
        "IA Cores Power [W]",
    ],
)
querying_runs_data = compute_total_energy_per_run(querying_experiments_data)

In [10]:
query_db_variation_runs = get_variation_runs_data(
    querying_runs_data,
    ["gte_base_cqadupstack_webmasters_milvus"],
    ["gte_base_cqadupstack_webmasters_qdrant"],
    ["gte_base_cqadupstack_webmasters_weaviate"],
    "database",
    ["milvus", "qdrant", "weaviate"],
)

query_db_variation_runs["IA Cores and DRAM Energy [J]"] = (
    query_db_variation_runs["IA Cores Energy [J]"] + query_db_variation_runs["DRAM Energy [J]"]
)

In [11]:
query_db_variation_runs.groupby("database").aggregate(
    lambda x: str(round(x.mean(), 2)) + " ± " + str(get_ci_deviation(x))
)

,IA Cores Energy [J],DRAM Energy [J],duration [s],IA Cores and DRAM Energy [J]
database,,,,
milvus,155.71 ± 3.45,9.7 ± 0.22,9.93 ± 0.11,165.41 ± 3.67
qdrant,155.37 ± 9.42,11.17 ± 0.65,8.91 ± 0.18,166.54 ± 10.07
weaviate,32.81 ± 0.42,2.29 ± 0.03,2.14 ± 0.02,35.1 ± 0.45


In [12]:
compute_wilcoxon(
    query_db_variation_runs, "weaviate", "milvus", "IA Cores and DRAM Energy [J]", "database"
)

WilcoxonResult(statistic=np.float64(0.0), pvalue=np.float64(0.0009765625))

In [13]:
compute_wilcoxon(
    query_db_variation_runs, "milvus", "qdrant", "IA Cores and DRAM Energy [J]", "database"
)

WilcoxonResult(statistic=np.float64(24.0), pvalue=np.float64(0.384765625))

In [14]:
compute_wilcoxon(
    query_db_variation_runs, "qdrant", "milvus", "IA Cores and DRAM Energy [J]", "database"
)

WilcoxonResult(statistic=np.float64(31.0), pvalue=np.float64(0.65234375))

In [15]:
compute_kruskal_wallis(
    query_db_variation_runs, "database", ["qdrant", "milvus"], "IA Cores and DRAM Energy [J]"
)

KruskalResult(statistic=np.float64(0.5714285714285694), pvalue=np.float64(0.4496917979688917))

In [16]:
compute_wilcoxon(query_db_variation_runs, "weaviate", "milvus", "duration [s]", "database")

WilcoxonResult(statistic=np.float64(0.0), pvalue=np.float64(0.0009765625))

In [17]:
compute_wilcoxon(query_db_variation_runs, "qdrant", "milvus", "duration [s]", "database")

WilcoxonResult(statistic=np.float64(0.0), pvalue=np.float64(0.0009765625))